In [5]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def compute_projection_matrix(normal):
    """
    Computes the 3x3 projection matrix that projects a 3D point onto a plane 
    with the given normal vector.
    """
    normal = normal / np.linalg.norm(normal)  # Normalize the normal vector
    P = np.eye(3) - np.outer(normal, normal)  # Projection matrix
    print(f"P: {P}")
    return P


In [6]:
def find_basis_vectors(normal):
    """
    Finds two orthogonal basis vectors in the plane perpendicular to the given normal.
    """
    normal = normal / np.linalg.norm(normal)
    
    # Pick an arbitrary vector not parallel to the normal
    if abs(normal[0]) < abs(normal[1]):
        u = np.array([1, 0, 0])
    else:
        u = np.array([0, 1, 0])
    
    # Make u orthogonal to normal
    u = u - np.dot(u, normal) * normal
    u = u / np.linalg.norm(u)
    
    # Compute v as cross product of normal and u
    v = np.cross(normal, u)
    v = v / np.linalg.norm(v)
    
    return u, v

In [7]:
class Sphere():
    def __init__(self, coordinates, radius, points):
        """Helper function to plot a sphere."""
        u = np.linspace(0, 2 * np.pi, points)
        v = np.linspace(0, np.pi, points)
        self.x = coordinates[0] + radius * np.outer(np.cos(u), np.sin(v))
        self.y = coordinates[1] + radius * np.outer(np.sin(u), np.sin(v))
        self.z = coordinates[2] + radius * np.outer(np.ones(np.size(u)), np.cos(v))

    def plot_sphere(self, ax, alpha = 1.0 ,label = 'Sphere', c='b'):
        ax.plot_wireframe(self.x, self.y, self.z, alpha=alpha, label=label,color=c)

    def get_x_y_z(self):
        return self.x, self.y, self.z
        

In [8]:
%matplotlib tk
class PointProjector():
    def __init__(self,normal):
        self.normal = normal
        self.P = compute_projection_matrix(normal)
        self.u, self.v = find_basis_vectors(normal)  # Get 2D basis vectors
    
    def change_basis(self, projected_point):
        
        x_2d = np.dot(projected_point, self.u)
        y_2d = np.dot(projected_point, self.v)
        arr_2d = np.array([x_2d, y_2d])
        return arr_2d

    def project_point_to_2d(self, point):
        projected_point = self.P @ point  # Apply projection
        return projected_point

    def get_u_v_P(self):
        return self.u, self.v, self.P
    
    def plot_plane(self, ax, projected_3d):
        # Define plane grid
        normal = self.normal
        d = -np.dot(normal, projected_3d)
        xx, yy = np.meshgrid(np.linspace(-10, 10, 10), np.linspace(-10, 10, 10))
        zz = (-normal[0] * xx - normal[1] * yy - d) / normal[2]

        # Plot the plane
        ax.plot_surface(xx, yy, zz, color='cyan', alpha=0.5)
    

def do_stuff():
    # Example usage
    normal = np.array([2, 2, 2])  # Example normal vector
    point = np.array([3, 4, 5])   # Example 3D point
    sphereC = np.array([4,-2,4])
    sphereR = 1.5
    sphereC2 = np.array([6,-3,6])
    sphereR2 = 1.5
    spherePoints = 50
    S = Sphere(sphereC,sphereR, spherePoints)
    S2 = Sphere(sphereC2,sphereR2, spherePoints)
    X,Y,Z = S.get_x_y_z()
    X2,Y2,Z2 = S2.get_x_y_z()

    PP = PointProjector(normal)
    u,v,P = PP.get_u_v_P()

    # Compute projection
    projected_3d = PP.project_point_to_2d(point)
    projected_2d = PP.change_basis(projected_3d)


    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(111, projection='3d')

    PP.plot_plane(ax, projected_3d)
    S.plot_sphere(ax, alpha = 0.5,c='r')
    S2.plot_sphere(ax, alpha = 0.5,c='b')

    # Plot points
    ax.scatter(*projected_3d, color='g', label='Projected Point', s=100)
    ax.scatter(*point, color='r', label='Original Point', s=100)

    # Draw projection line
    ax.plot([point[0], projected_3d[0]], [point[1], projected_3d[1]], [point[2], projected_3d[2]], 'k--', label='Projection Line')
    for i in range(spherePoints):
        for j in range(spherePoints):
            p = PP.project_point_to_2d(np.array([X[i,j], Y[i,j], Z[i,j]]))
            p2 = PP.project_point_to_2d(np.array([X2[i,j], Y2[i,j], Z2[i,j]]))
            ax.scatter(p[0], p[1], p[2], color='g', s=1, alpha = 0.3)
            ax.scatter(p2[0], p2[1], p2[2], color='y', s=1, alpha = 0.3)


    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.legend()
    plt.show()

    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(111)

    ax.scatter(*projected_2d, color='g', label='Projected Point', s=100)
    for i in range(spherePoints):
        for j in range(spherePoints):
            p = PP.project_point_to_2d(np.array([X[i,j], Y[i,j], Z[i,j]]))
            p = PP.change_basis(p)
            p2 = PP.project_point_to_2d(np.array([X2[i,j], Y2[i,j], Z2[i,j]]))
            p2 = PP.change_basis(p2)
            ax.scatter(p[0], p[1], color='g', s=1, alpha = 0.3)
            ax.scatter(p2[0], p2[1], color='y', s=1, alpha = 0.3)
    plt.axis('equal')
    plt.show()

do_stuff()

P: [[ 0.66666667 -0.33333333 -0.33333333]
 [-0.33333333  0.66666667 -0.33333333]
 [-0.33333333 -0.33333333  0.66666667]]


invalid command name "138418580375296idle_draw"
    while executing
"138418580375296idle_draw"
    ("after" script)
